In [2]:
from tqdm import tqdm_notebook

train = '389125467'
test = '215694783'

class CupCircle:

    def __init__(self, cups):
        
        cups = list(map(int, cups))
        
        data = {
            value: next_value
            for value, next_value
            in zip(list(cups), list(cups)[1:])}
        
        data[list(cups)[-1]] = list(cups)[0]
            
        self.series = pd.Series(
            data = data,
            name = 'next')
        
        self.current = self.series.index[0]
        self.bar = tqdm_notebook()
        
        self.min_value = self.series.min()
        self.max_value = self.series.max()
        
    def move(self):

        # print(f'cups: {self.cups_str(self.current)}')
        
        removed_cups = list(itertools.islice(self.next_iter(self.current), 1, 4))
        # print('pick up: ' + ' '.join(map(str, removed_cups)))

        self.series[self.current] = self.series[removed_cups[-1]]

        destination = self.current - 1

        while True:
            
            if destination < self.min_value:
                destination = self.max_value
            
            if destination not in removed_cups:
                break
                
            destination -= 1

        # print(f'destination: {destination}')
        
        self.series[removed_cups[-1]] = self.series[destination]
        self.series[destination]      = removed_cups[0]
            
        self.current = self.series[self.current]    
            
        self.bar.update()

        return self
    
    def next_iter(self, start):
        
        current = start
        yield start
        
        while True:
            current = self.series[current]
            yield current
            
    def cups_str(self, start):
        
        values = []
        for value in self.next_iter(start):
            if len(values) > 0 and value == start:
                break
            values.append(value)
            
        return ' '.join(map(str, values))

    def simulate(self, n):
        for _ in range(n):
            self.move()
            
        return self
    
    def results(self):
        return int(self.cups_str(1).replace(' ', '')[1:])
        
self = CupCircle(cups=train)
assert self.simulate(10).results() == 92658374

self = CupCircle(cups=train)
assert self.simulate(100).results() == 67384529

self = CupCircle(cups=test)
self.simulate(100).results()

/home/ec2-user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


46978532

In [3]:
class MillionCupCircle(CupCircle):
    
    def __init__(self, cups, max_value=1000000):
        
        super().__init__(cups=cups)
        
        additions = list(range(self.series.max() + 1, max_value + 1))
        
        self.series.iloc[-1] = additions[0]
        
        data = {
            value: next_value
            for value, next_value
            in zip(additions, additions[1:])}
        data[max_value] = self.series.index[0]
        
        self.series = self.series.append(pd.Series(data))
        
        self.min_value = self.series.min()
        self.max_value = self.series.max()
            
    def results(self):
        value_1 = self.series[1]
        value_2 = self.series[value_1]
        return value_1 * value_2
            
self = MillionCupCircle(cups=train)
assert self.simulate(10000000).results() == 149245887792

self = MillionCupCircle(cups=test)
self.simulate(10000000).results()

/home/ec2-user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


163035127721